In [6]:
#renders matplotlib plots in the notebook
%matplotlib notebook 

#enables tab autocomplete feature
%config IPCompleter.greedy=True 

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

## Random Variable
A random variable is a function that maps the sample space S into a subset of the real line.

X - Random variable - known deterministic function, which maps $s_i$ to $x_i$ <br/>

$X :: S \rightarrow R$, $s_i \leftarrow S, x_i \leftarrow Z$

the discrete random variable, which is one that takes on a finite or countably infinite number of values.

$S$ - Original sample space <br/>
$S_X$ - new sample space

### Probability Mass Function

$px[x_i] = P[X(s) = x_i]$

probability mass function maps the real numbers x_i to their probabilities

$P :: S \rightarrow [0,1]$

$X :: S \rightarrow Z$

$px :: Z \rightarrow [0,1]$

In summary, the probability mass function is the probability that the random variable X takes on the value x_i for each possible x_i.

**Convention**: $X(s)$ is represented as just $X$

![random variable and PMF](files/random_variable_and_PMF.jpg)

### Bernoulli PMF

Bernoulli experiment has only 2 outcomes, success and failure.

$X  \sim  Ber(p)$ means $X$ is distributed as per the Bernoulli probability mass function where probability of success is p.

### Binomial PMF

$bin(M,k) = \left ( {M \atop k} \right )p^k(1-p)^{M-k}$

Since $bin(M,k)$ is PMF does $\sum_{k=0}^{k=M} bin(M,k) = 1$?

In [12]:
from utils import utils

M = 50
p = 0.5

K = np.arange(0,M+1)
S = np.sum([utils.binomial_probability(M, k, p) for k in K])
print(S)
assert np.abs(S - 1) < 1e-6

1.0


### Geometric PMF
page 113 (5.7)

The probability that the first success occurs at experiment / trial k

$p_X[k] = (1-p)^{(1-k)}p^k$

### Poisson PMF
page 113 (5.8)

$p_X[k] = e^{-\lambda}  \lambda^k / k!$


## Random Variable Transformation

![random variable transformation](files/transformed_random_variable.jpg)


If the transformation is one-to-one then finding the new PMF is straight forward. For every element of the transformed set, the probability is the same it's image in the original set.

If the transformation is many-to-one then $P_Y(y_i)$ is the summation of PMF of all $x$ that map to $y_i$

$$P_Y[y] = \sum_{\forall g(x_i) = y}^{}P_X[x_i]$$

## Cumulative Distributive Function